## Introduction to PyTorch

### PyTorch Tensors

In [1]:
import torch
import math # constant status

### Creating Tensors

In [5]:
x = torch.tensor([[5.5, 3],[1, 2]])
print(type(x))
print(x)

x = torch.empty(3, 4) # 只是分配了内存，但是没有初始化，所以值是隨機的
print(type(x))
print(x)

<class 'torch.Tensor'>
tensor([[5.5000, 3.0000],
        [1.0000, 2.0000]])
<class 'torch.Tensor'>
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


Let's unpack what we just did:
* We created a tensor using one of the numerous factory methods attached to the torch module.
* The tensor itself is 2-dimensional, having 3 rows and 4 columns.
* The type of the object returned is `torch. Tensor`, which is an alias for `torch.FloatTensor` ; by default, PyTorch tensors are populated with 32-bit floating point numbers. (More on data types below.)
* You will probably see some random-looking values when printing your tensor. The `torch. empty()` call allocates memory for the tensor, but does not initialize it with any values - so what you're seeing is whatever was in memory at the time of allocation.

A brief note about tensors and their number of dimensions, and terminology:
* You will sometimes see a 1-dimensional tensor called a vector.
* Likewise, a 2-dimensional tensor is often referred to as a matrix.
* Anything with more than two dimensions is generally just called a tensor.
More often than not, you'll want to initialize your tensor with some value. Common cases are all zeros, all ones, or random values, and the torch module provides factory methods for all of these:

In [6]:
zeros = torch.zeros(2, 3)
print(zeros)

ones = torch.ones(2, 3)
print(ones)

torch.manual_seed(1729)
random = torch.rand(2, 3)
print(random)


tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.3126, 0.3791, 0.3087],
        [0.0736, 0.4216, 0.0691]])


The factory methods all do just what you'd expect - we have a tensor full of zeros, another full of ones, and another with random values between 0 and 1.  



**Random Tensors and Seeding**  
Speaking of the random tensor, did you notice the call to `torch-manual seed()` immediately preceding it? Initializing tensors, such as a model's learning weights, with random values is common but there are times - especially in research settings - where you'll want some assurance of the reproducibility of your results. Manually setting your random number generator's seed is the way to do this. Let's look more closely:

In [21]:
torch.manual_seed(1729)
random1 = torch.rand(2, 3)
print(random1)

random2 = torch.randn(2, 3)
print(random2)

torch.manual_seed(1729)
random3 = torch.randn(2, 3)
print(random3)


random4 = torch.rand(2, 3)
print(random4)

tensor([[0.3126, 0.3791, 0.3087],
        [0.0736, 0.4216, 0.0691]])
tensor([[ 1.0757, -1.2086, -0.6922],
        [ 2.0419, -1.8508,  2.1626]])
tensor([[-1.1257, -0.0057, -1.3975],
        [ 1.4364, -0.1068, -0.8413]])
tensor([[0.6128, 0.1519, 0.0453],
        [0.5035, 0.9978, 0.3884]])


What you should see above is that `random1` and `random3` carry identical values, as do `random2` and `random4`. Manually setting the RNG's seed resets it, so that identical computations depending on random number should, in most settings, provide identical results.  
For more information, see the PyTorch documentation on reproducibility.



**Tensor Shapes**  
Often, when you're performing operations on two or more tensors, they will need to be of the same shape - that is, having the same number of dimensions and the same number of cells in each dimension. For that, we have the `torch.*_like()` methods:

In [22]:
x = torch.empty(2, 2, 3)
print(x.shape)
print(x)

empty_like_x = torch.empty_like(x)
print(empty_like_x.shape)
print(empty_like_x)

zeros_like_x = torch.zeros_like(x)
print(zeros_like_x.shape)
print(zeros_like_x)

ones_like_x = torch.ones_like(x)
print(ones_like_x.shape)
print(ones_like_x)

rand_like_x = torch.rand_like(x)
print(rand_like_x.shape)
print(rand_like_x)

torch.Size([2, 2, 3])
tensor([[[ 2.5041e-32,  9.3467e-43, -0.0000e+00],
         [ 1.6895e+00, -2.0000e+00,  1.6543e+00]],

        [[ 0.0000e+00,  1.3972e+00,  2.0000e+00],
         [ 1.7108e+00,  0.0000e+00,  1.3881e+00]]])
torch.Size([2, 2, 3])
tensor([[[ 2.5039e-32,  9.3467e-43,  1.0842e-19],
         [ 9.6553e-01,  1.0842e-19,  1.9247e+00]],

        [[-3.6893e+19,  1.9296e+00,  1.0842e-19],
         [ 1.4636e+00,  2.0000e+00,  1.8353e+00]]])
torch.Size([2, 2, 3])
tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])
torch.Size([2, 2, 3])
tensor([[[1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.]]])
torch.Size([2, 2, 3])
tensor([[[0.6929, 0.1703, 0.1384],
         [0.4759, 0.7481, 0.0361]],

        [[0.5062, 0.8469, 0.2588],
         [0.2707, 0.4115, 0.6839]]])


The first new thing in the code cell above is the use of the `.shape` property on a tensor. This property contains a list of the extent of each dimension of a tensor - in our case, `x` is a three-dimensional tensor with shape 2x2x3.  
Below that, we call the `.empty_like()`, `.zeros_like()`, `.ones_like()` , and `.rand_ like()` methods. Using the `.shape` property, we can verify that each of these methods returns a tensor of identical dimensionality and extent.  
The last way to create a tensor that will cover is to specify its data directly from a PyTorch collection:

In [23]:
some_constants = torch.tensor([[3.1415926, 2.71828],[1.61803, 0.0072897]])
print(some_constants)

some_integers = torch.tensor((2, 3, 5, 7, 11, 13, 17, 19))
print(some_integers)

more_integers = torch.tensor(([2, 4, 6], [3, 6, 9]))
print(more_integers)

tensor([[3.1416, 2.7183],
        [1.6180, 0.0073]])
tensor([ 2,  3,  5,  7, 11, 13, 17, 19])
tensor([[2, 4, 6],
        [3, 6, 9]])


Using `torch.tensor()` is the most straightforward way to create a tensor if you already have data in a Python tuple or list. As shown above, nesting the collections will result in a multi-dimensional tensor.  
Note: `torch.tensor()` creates a copy of the data.



**Tensor Data Types**  
Setting the datatype of a tensor is possible a couple of ways:

In [24]:
a = torch.ones((2, 3), dtype=torch.int16)
print(a)

b = torch.rand((2, 3), dtype=torch.float32) * 20
print(b)

c = b.to(torch.int64)
print(c)

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int16)
tensor([[ 1.4051, 10.2103, 18.9010],
        [ 4.7172,  3.9587,  6.6549]])
tensor([[ 1, 10, 18],
        [ 4,  3,  6]])


The simplest way to set the underlying data type of a tensor is with an optional argument at creation time. In the first line of the cell above, we set `dtype=torch.intl6` for the tensor `a`. When we print `a`, we can see that it's full of `1` rather than `1.` - Python's subtle cue that this is an integer type rather than floating point.  
Another thing to notice about printing a is that, unlike when we left `dtype` as the default (32-bit floating point), printing the tensor also specifies its `dtype`.  
You may have also spotted that we went from specifying the tensor's shape as a series of integer arguments, to grouping those arguments in a tuple. This is not strictly necessary - PyTorch will take a series of initial, unlabeled integer arguments as a tensor shape - but when adding the optional arguments, it can make your intent more readable.  
The other way to set the datatype is with the `.to()` method. In the cell above, we create a random floating point tensor `b` in the usual way. Following that, we create `c` by converting `b` to a 32-bit integer with the `.to()` method. Note that `c` contains all the same values as `b`, but truncated to integers.  
Available data types include:  
* torch.bool
* torch.int8
* torch.uint8
* torch.int16
* torch.int32
* torch.int64
* torch.half
* torch.tloat
* torch.double
* torch.bfloat

**Math & Logic with PyTorch Tensors**  


